# Mining Massive Datasets: Solutions to Problem Set 3

Duc Anh, Phi 3550091

Mustafa, Ibrahim 3284705

Amritpal, Kaur 3532964

## Exercise 1

### a)


In [1]:
import numpy as np

util_matrix = np.array([[4,5,0,5,1,0,3,2], [0,3,4,3,1,2,1,0],[2,0,1,3,0,4,5,3],[1,0,1,0,4,0,4,2]])

def avg_without_blank(x):
    n = np.count_nonzero(x)
    return np.sum(x)/n

def pearson_sim(x,y):
    x_b = x.astype(bool).astype(int)
    y_b = y.astype(bool).astype(int)
    indeces = np.nonzero(np.bitwise_and(x_b, y_b))
    x_centered = x-avg_without_blank(x)
    y_centered = y-avg_without_blank(y)
    x_reduced = x_centered[indeces]
    y_reduced = y_centered[indeces]
    
    res = np.dot(x_reduced, y_reduced) / (np.linalg.norm(x_reduced)*np.linalg.norm(y_reduced))
    return res

def rating(util_matrix, N, item, user):
    similarities = []
    for i in range(util_matrix.shape[0]):
        if i != user:
            similarities.append(pearson_sim(util_matrix[user], util_matrix[i]))
        else:
            similarities.append(-2)
    similarities = np.array(similarities)
    
    most_similar = similarities.argsort()[::-1][:N] 
    
    numer = 0
    denom = 0
    for i in range(N):
        sim_user_i = similarities[most_similar[i]]
        numer += sim_user_i*util_matrix[most_similar[i]][item]
        denom += sim_user_i
    rating = numer/denom
    return rating

N = 2
item = 4
user = 2

print("predicted rating for item e: ", rating(util_matrix, N, item, user))

predicted rating for item e:  5.140960958442029


### b)

In [2]:
util_matrix = util_matrix.T
N = 2
item = 2
user = 4

print("predicted rating for item e: ", rating(util_matrix, N, item, user))

predicted rating for item e:  4.406435469853564


## Exercise 2

We first implement the Stochastic Gradient Descent (SGD) method:

In [3]:
def gradient_Q_i(d, x_i, y_i, w):
    res = np.zeros(d)
    for k in range(d):
        for j in range(d):
            res[k] += w[j]*(x_i**j)
        res[k] = 2*(res[k]-y_i)*(x_i**k)
    return res

def Q_i(d, x_i, y_i, w):
    res = 0
    for j in range(d):
        res += w[j]*(x_i**j)
    res = (res-y_i)**2
    return res
        
def Q(d, points, w):
    res = 0
    for point in points:
        res += Q_i(d, point[0], point[1], w)
    return res

def SGD(d, points, alpha=0.00001, eps=0.1):
    w = np.random.rand(d)
    while True:
        err_before = Q(d, points, w)
        for point in points:
            w = w - alpha*gradient_Q_i(d, point[0], point[1], w)
        err_after = Q(d, points, w)
        if np.abs(err_before - err_after) < eps:
            break
    return w


Then we define some polynomial, construct slightly perturbed points and run the SGD function on those points:

In [4]:
def poly(x):
    return -1-2*x+x**2+0.5*(x**3)

new_points = []

x_points = np.linspace(-5, 5, 1000)
y_points = poly(x_points)

for i in range(x_points.shape[0]):
    perturbation = np.random.rand()*2-1
    new_points.append((x_points[i], y_points[i]+perturbation))


print(SGD(4, new_points))

[-0.94505448 -1.94480143  0.99607806  0.4955918 ]


The resulting polynomial is within a 0.6 range of the original polynomial.